In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
# from tensorflow.keras.applications.densenet import DenseNet121
# from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
# from tensorflow.keras.applications.resnet_v2 import ResNet101V2

# Data Preparation

In [2]:
train_data_dir = "flowers/train"
val_data_dir = "flowers/val"
test_data_dir = "flowers/test"

#change according to model 
image_size = 224
BATCH_SIZE = 32

In [4]:
class_subset = sorted(f for f in os.listdir(train_data_dir) if not f.startswith('.'))
# sorted(os.listdir(train_data_dir))
print(class_subset)

['daisy', 'dandelion', 'lily', 'orchid', 'rose', 'sunflower', 'tulip']


In [5]:
#central crop
def crop_center_square(image):
    img_height, img_width = image.shape[0:2]
    min_dim = min(img_height, img_width)
    start_x = (img_width // 2) - (min_dim // 2)
    start_y = (img_height // 2) - (min_dim // 2)
    
    img = image[start_y : start_y + min_dim, start_x : start_x + min_dim]
    img = cv2.resize(img, (image_size, image_size))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    return img

In [6]:
train_generator = ImageDataGenerator(rescale=1/255.,
                                     featurewise_center= False,
                                     samplewise_center= False,
                                     featurewise_std_normalization= False,
                                     samplewise_std_normalization=False,
                                     rotation_range= 10,     # 0-180
                                     zca_whitening=False,
                                     zoom_range=0.1,         # Randomly zoom image
                                     width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
                                     height_shift_range=0.2, # randomly shift images vertically (fraction of total height)
                                     horizontal_flip=True,   # randomly flip images
                                     vertical_flip=False,
                                     validation_split=0.0,
                                     preprocessing_function=crop_center_square)

val_generator  = ImageDataGenerator(rescale=1/255.,
                                    preprocessing_function=crop_center_square)

test_generator = ImageDataGenerator(rescale=1/255.,
                                    preprocessing_function=crop_center_square)

In [7]:
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(image_size, image_size),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=21)

validgen = val_generator.flow_from_directory(val_data_dir,
                                             target_size=(image_size, image_size),
                                             classes=class_subset,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True,
                                             seed=21)

testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(image_size, image_size),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=21)

class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

Found 8069 images belonging to 7 classes.
Found 2306 images belonging to 7 classes.
Found 1156 images belonging to 7 classes.


In [16]:
# VGG16
base_model = VGG16(weights='imagenet',
                   include_top= False, 
                   input_shape=(image_size,image_size,3),
                   pooling='avg')

base_model.trainable = False

model = Sequential()
model.add(base_model)
# model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(len(class_subset),activation="softmax"))

model.compile(optimizer=optimizers.Adam(learning_rate = 1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

In [ ]:
# InceptionV3
base_model = InceptionV3(input_shape = (image_size, image_size, 3), 
                         include_top = False, 
                         weights = 'imagenet',
                         pooling='avg')

base_model.trainable = False

model = Sequential()
model.add(base_model)
# model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(class_subset),activation="softmax"))

model.compile(optimizer=optimizers.Adam(learning_rate = 1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

In [ ]:
# InceptionResNetV2
base_model = InceptionResNetV2(input_shape = (image_size, image_size, 3), 
                               include_top = False, 
                               weights = "imagenet", 
                               pooling='avg')

base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5,activation="softmax"))

model.compile(optimizer=optimizers.Adam(learning_rate = 1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

In [ ]:
# DenseNet169
base_model = DenseNet121(weights='imagenet',
                         include_top= False, 
                         input_shape=(image_size,image_size,3), 
                         pooling='avg')

base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(5,activation="softmax"))

model.compile(optimizer=optimizers.Adam(learning_rate = 1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

In [ ]:
# MobileNetV2
base_model = MobileNetV2(weights='imagenet',
                         include_top= False, 
                         input_shape=(image_size,image_size,3), 
                         pooling='avg')

base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(5,activation="softmax"))

model.compile(optimizer=optimizers.Adam(learning_rate = 1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

In [ ]:
# ResNet50
base_model = ResNet101V2(weights='imagenet',
                         include_top= False, 
                         input_shape=(image_size,image_size,3), 
                         pooling='avg')

base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(5,activation="softmax"))

model.compile(optimizer=optimizers.Adam(learning_rate = 1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import ReduceLROnPlateau

# # EarlyStopping
early_stop = EarlyStopping(monitor='val_accuracy',
                           patience=3,
                           restore_best_weights=True,
                           mode='max')

red_lr = ReduceLROnPlateau(monitor='val_loss',
                           factor=0.1,
                           epsilon=0.0001,
                           patience=2,
                           verbose=1)

weight_file = 'vgg16_model.hdf5'
# weight_file = 'inceptionv3_model.hdf5'
# weight_file = 'inception_resnet_model.hdf5'
# weight_file = 'densenet121_model.hdf5'
# weight_file = 'mobilenetv2_model.hdf5'
# weight_file = 'resnet101_model.hdf5'

best_checkpoint = ModelCheckpoint(weight_file,
                                  save_weights_only=False, 
                                  save_best_only=True,
                                  monitor='val_accuracy',
                                  mode='max',
                                  verbose=1)

history_file = 'vgg16_history.csv'
# history_file = 'inceptionv3_history.csv'
# history_file = 'inception_resnet_history.csv'
# history_file = 'densenet121_history.csv'
# history_file = 'mobilenetv2_history.csv'
# history_file = 'resnet101_history.csv'

history_logger = CSVLogger(history_file, separator=",", append=True)

n_epochs = 10

history = model.fit(traingen,
                    batch_size=BATCH_SIZE,
                    epochs=n_epochs,
                    validation_data=validgen,
                    steps_per_epoch=traingen.samples // BATCH_SIZE,
                    validation_steps=validgen.samples // BATCH_SIZE,
                    callbacks=[best_checkpoint, history_logger, red_lr, early_stop],
                    verbose=1)

In [18]:
from sklearn.metrics import classification_report

preds = model.predict(testgen)
preds = np.argmax(preds, axis=1)

report = classification_report(testgen.classes, preds)
print(report)

In [19]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(testgen.classes, preds)
print("VGG16 Model Accuracy: {:.2f}%".format(acc * 100))
# print("InceptionV3 Model Accuracy: {:.2f}%".format(acc * 100))
# print("Inception_ResNet Model Accuracy: {:.2f}%".format(acc * 100))
# print("Densenet121 Model Accuracy: {:.2f}%".format(acc * 100))
# print("MobilenetV2 Model Accuracy: {:.2f}%".format(acc * 100))
# print("Resnet101 Model Accuracy: {:.2f}%".format(acc * 100))

In [20]:
plt.plot(history.epoch, history.history['accuracy'])
plt.plot(history.epoch, history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.legend(['train', 'validation'])
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.figure()

plt.plot(history.epoch, history.history['loss'])
plt.plot(history.epoch, history.history['val_loss'])
plt.title('Model Loss')
plt.legend(['train', 'validation'])
plt.xlabel('No. of Epochs')
plt.ylabel('Loss')
plt.show()

# Model Finetuning

In [23]:
# VGG16, InceptionV3, Inception_ResNet, Densenet121, MobileNetV2, EfficientNet, Resnet101
# freeze all layers except from layer 15 onwards
base_model.trainable = True

for layer in base_model.layers[:-15]:
    layer.trainable=False

model = Sequential()
model.add(base_model)
# model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(len(class_subset),activation="softmax"))
model.summary()

# Recompile the model
model.compile(optimizer=optimizers.Adam(learning_rate = 1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [24]:
n_epochs = 10

fine_tune_history = model.fit(traingen,
                              batch_size=BATCH_SIZE,
                              epochs=n_epochs, 
                              validation_data=validgen,
                              steps_per_epoch=traingen.samples // BATCH_SIZE,
                              validation_steps=validgen.samples // BATCH_SIZE,
                              callbacks=[best_checkpoint, history_logger, red_lr, early_stop],
                              verbose=1)

In [13]:
from sklearn.metrics import classification_report

class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

preds = model.predict(testgen)
preds = np.argmax(preds, axis=1)

report = classification_report(testgen.classes, preds)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       176
           1       0.99      0.99      0.99       150
           2       0.98      0.99      0.99       162
           3       0.99      0.99      0.99       186
           4       0.99      0.99      0.99       149
           5       0.98      0.98      0.98       172
           6       1.00      0.99      0.99       161

    accuracy                           0.99      1156
   macro avg       0.99      0.99      0.99      1156
weighted avg       0.99      0.99      0.99      1156



In [27]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(testgen.classes, preds)
print("VGG16 Model Accuracy: {:.2f}%".format(acc * 100))
# print("InceptionV3 Model Accuracy: {:.2f}%".format(acc * 100))
# print("Inception_ResNet Model Accuracy: {:.2f}%".format(acc * 100))
# print("Densenet121 Model Accuracy: {:.2f}%".format(acc * 100))
# print("MobileNetV2 Model Accuracy: {:.2f}%".format(acc * 100))
# print("ResNet101 Model Accuracy: {:.2f}%".format(acc * 100))

In [28]:
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

cm  = confusion_matrix(testgen.classes, preds)

plt.figure()
plot_confusion_matrix(cm,figsize=(12,8),cmap=plt.cm.Blues)
plt.xticks(range(len(class_indices)), class_indices.values(), fontsize=16)
plt.yticks(range(len(class_indices)), class_indices.values(), fontsize=16)
plt.xlabel('Predicted Label',fontsize=18)
plt.ylabel('True Label',fontsize=18)
plt.show()

# Single Image Inference

In [29]:
from keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model

In [30]:
import matplotlib.pyplot as plt

filename = '../input/flowerdatasets/flowers/test/orchid/1012229979_3ecc9f9859_o.jpg'
image = load_img(filename, target_size=(image_size, image_size))
plt.imshow(image)
plt.show()

In [31]:
import cv2
import numpy as np
import time

image_size = 224 

img = cv2.imread(filename)
img = cv2.resize(img,(image_size,image_size))
img = img/255
img = np.reshape(img,[1,image_size,image_size,3])

model = load_model('vgg16_model.hdf5')
# model = load_model('inceptionv3_model.hdf5')
# model = load_model('inception_resnet_model.hdf5')
# model = load_model('densenet121_model.hdf5')
# model = load_model('mobilenetv2_model.hdf5')
# model = load_model('resnet101_model.hdf5')

start_time = time.time()
prediction=model.predict(img) 
prediction=np.argmax(prediction,axis=1)[0]
end_time = time.time()

print("Predicted class: {}, time {:.3f}s".format(class_indices[prediction], end_time-start_time))